In [4]:
import time 
import requests
import operator
import numpy as np
from __future__ import print_function

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [9]:
_region = 'westcentralus' #Here you enter the region of your subscription
_url = 'https://{}.api.cognitive.microsoft.com/vision/v1.0/analyze'.format(_region)
_key = '01c8333210854059a6c2df093bf1b284'
_maxNumRetries = 10

In [12]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json() ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                    print(response.elapsed.total_seconds())
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [17]:
image_path = '/Users/balli/Downloads/ss2.jpg'
with open( image_path , 'rb' ) as f:
    data = f.read()
    
# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories,Description'} 

headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None

result = processRequest( json, data, headers, params )

if result is not None:
    print(result)

5.289685
{'categories': [{'name': 'outdoor_', 'score': 0.01953125, 'detail': {'landmarks': []}}, {'name': 'outdoor_street', 'score': 0.35546875, 'detail': {'landmarks': []}}], 'description': {'tags': ['building', 'outdoor', 'street', 'scene', 'sidewalk', 'road', 'city', 'wet', 'rain', 'parked', 'water', 'side', 'walking', 'lined', 'man', 'rainy', 'umbrella', 'hydrant', 'fire', 'covered', 'red', 'holding', 'brick', 'riding', 'old', 'bus', 'tall', 'sign', 'traffic', 'people', 'standing', 'group'], 'captions': [{'text': 'a rainy city street', 'confidence': 0.8684569279238283}]}, 'color': {'dominantColorForeground': 'Grey', 'dominantColorBackground': 'Grey', 'dominantColors': ['Grey'], 'accentColor': '976834', 'isBwImg': False}, 'requestId': '4bd07c1a-4062-4421-8cfd-6a2668015242', 'metadata': {'height': 420, 'width': 631, 'format': 'Jpeg'}}
